## Оптимизация выполнения кода, векторизация, Numba

Материалы:
* Макрушин С.В. Лекция 3: Оптимизация выполнения кода, векторизация, Numba
* IPython Cookbook, Second Edition (2018), глава 4
* https://numba.pydata.org/numba-doc/latest/user/5minguide.html

In [1]:
import pandas as pd

## Задачи для совместного разбора

1. Сгенерируйте массив `A` из `N=1млн` случайных целых чисел на отрезке от 0 до 1000. Пусть `B[i] = A[i] + 100`. Посчитайте среднее значение массива `B`.

2. Создайте таблицу 2млн строк и с 4 столбцами, заполненными случайными числами. Добавьте столбец `key`, которые содержит элементы из множества английских букв. Выберите из таблицы подмножество строк, для которых в столбце `key` указаны первые 5 английских букв.

## Лабораторная работа 3

In [ ]:
# !pip install line_profiler

1. В файлах `recipes_sample.csv` и `reviews_sample.csv` (__ЛР 2__) находится информация об рецептах блюд и отзывах на эти рецепты соответственно. Загрузите данные из файлов в виде `pd.DataFrame` с названиями `recipes` и `reviews`. Обратите внимание на корректное считывание столбца(ов) с индексами. Приведите столбцы к нужным типам.

Реализуйте несколько вариантов функции подсчета среднего значения столбца `rating` из таблицы `reviews` для отзывов, оставленных в 2010 году.

A. С использованием метода `DataFrame.iterrows` исходной таблицы;

Б. С использованием метода `DataFrame.iterrows` таблицы, в которой сохранены только отзывы за 2010 год;

В. С использованием метода `Series.mean`.

Проверьте, что результаты работы всех написанных функций корректны и совпадают. Измерьте выполнения всех написанных функций.


In [71]:
parse_dates = ['submitted']
recipes = pd.read_csv('recipes_sample.csv',parse_dates=parse_dates)
recipes['year'] = pd.DatetimeIndex(recipes['submitted']).year
reviews = pd.read_csv('reviews_sample.csv',index_col = 0)
recipes.head()

,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients,year
0,george s at the cove black bean soup,44123,90,35193,2002-10-25,NaN,an original recipe created by chef scott meska...,18.0,2002
1,healthy for them yogurt popsicles,67664,10,91970,2003-07-26,NaN,my children and their friends ask for my homem...,NaN,2003
2,i can t believe it s spinach,38798,30,1533,2002-08-29,NaN,"these were so go, it surprised even me.",8.0,2002
3,italian gut busters,35173,45,22724,2002-07-27,NaN,my sister-in-law made these for us at a family...,NaN,2002
4,love is in the air beef fondue sauces,84797,25,4470,2004-02-23,4.0,i think a fondue is a very romantic casual din...,NaN,2004


**A. С использованием метода DataFrame.iterrows исходной таблицы;**

In [3]:
reviews['year'] = pd.DatetimeIndex(reviews['date']).year
reviews.head()

,user_id,recipe_id,date,rating,review,year
370476,21752,57993,2003-05-01,5,Last week whole sides of frozen salmon fillet ...,2003
624300,431813,142201,2007-09-16,5,So simple and so tasty! I used a yellow capsi...,2007
187037,400708,252013,2008-01-10,4,"Very nice breakfast HH, easy to make and yummy...",2008
706134,2001852463,404716,2017-12-11,5,These are a favorite for the holidays and so e...,2017
312179,95810,129396,2008-03-14,5,Excellent soup! The tomato flavor is just gre...,2008


In [5]:
def mean_a(data,column):
    row_iterator = data.iterrows()
    summ = 0
    count = 0 
    for i, row in row_iterator:
        summ += row[column]
        count +=1
    return summ/count
        

In [6]:
%%timeit
mean = mean_a(reviews,"rating")
mean

9.17 s ± 326 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


**Б. С использованием метода DataFrame.iterrows таблицы, в которой сохранены только отзывы за 2010 год;**

In [7]:
#создаем таблицу с рецептами за 2010 год
reviews_2010 = reviews.loc[reviews['year'] == 2010]

In [8]:
%%timeit
mean = mean_a(reviews_2010,"rating")
mean

856 ms ± 7.58 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


**В. С использованием метода Series.mean.**

In [9]:
def mean_c(data,column):
    return data[column].mean()

In [10]:
%%timeit
mean_c(reviews,'rating')

176 µs ± 8.75 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


2. Какая из созданных функций выполняется медленнее? Что наиболее сильно влияет на скорость выполнения? Для ответа использовать профайлер `line_profiler`. Сохраните результаты работы профайлера в отдельную текстовую ячейку и прокомментируйте результаты его работы.


In [57]:
#устанавливаем бибилиотеку
#pip install memory_profiler

In [11]:
%load_ext memory_profiler

In [38]:
%%writefile mean_a.py
def mean_a(data,column):
        row_iterator = data.iterrows()
        summ = 0
        count = 0 
        for i, row in row_iterator:
            summ += row[column]
            count +=1
        return summ/count

Overwriting mean_a.py


In [39]:
from mean_a import mean_a

In [41]:
%memit mean_a(reviews_2010,"rating")

peak memory: 139.18 MiB, increment: 0.56 MiB


In [42]:
%%writefile mean_c.py
def mean_c(data,column):
        return data[column].mean()

Overwriting mean_c.py


In [43]:
from mean_c import mean_c

In [44]:
%memit mean_c(reviews,"rating")

peak memory: 138.95 MiB, increment: 0.00 MiB


3. Вам предлагается воспользоваться функцией, которая собирает статистику о том, сколько отзывов содержат то или иное слово. Измерьте время выполнения этой функции. Сможете ли вы найти узкие места в коде, используя профайлер? Выпишите (словами), что в имеющемся коде реализовано неоптимально. Оптимизируйте функцию и добейтесь значительного (как минимум, на один порядок) прироста в скорости выполнения.

In [48]:
def get_word_reviews_count(df):
    word_reviews = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        recipe_id, review = row['recipe_id'], row['review']
        words = review.split(' ')
        for word in words:
            if word not in word_reviews:
                word_reviews[word] = []
            word_reviews[word].append(recipe_id)
    
    word_reviews_count = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        review = row['review']
        words = review.split(' ')
        for word in words:
            word_reviews_count[word] = len(word_reviews[word])
    return word_reviews_count

In [49]:
%%timeit
get_word_reviews_count(reviews)

28 s ± 1.24 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [60]:
def get_word_reviews_count2(df):
    word_reviews = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        recipe_id, review = row['recipe_id'], row['review']
        words = review.split(' ')
        for word in words:
            if word not in word_reviews:
                word_reviews[word] = 1
            else:
                word_reviews[word] +=1
    return word_reviews


In [61]:
%%timeit
get_word_reviews_count2(reviews)

14.2 s ± 1.27 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [72]:
reviews = reviews.reset_index().drop(['index'],axis=1)
reviews.head()

,user_id,recipe_id,date,rating,review
0,21752,57993,2003-05-01,5,Last week whole sides of frozen salmon fillet ...
1,431813,142201,2007-09-16,5,So simple and so tasty! I used a yellow capsi...
2,400708,252013,2008-01-10,4,"Very nice breakfast HH, easy to make and yummy..."
3,2001852463,404716,2017-12-11,5,These are a favorite for the holidays and so e...
4,95810,129396,2008-03-14,5,Excellent soup! The tomato flavor is just gre...


4. Напишите несколько версий функции `MAPE` (см. [MAPE](https://en.wikipedia.org/wiki/Mean_absolute_percentage_error)) для расчета среднего абсолютного процентного отклонения значения рейтинга отзыва на рецепт от среднего значения рейтинга по всем отзывам для этого рецепта. 
    1. Без использования векторизованных операций и методов массивов `numpy` и без использования `numba`
    2. Без использования векторизованных операций и методов массивов `numpy`, но с использованием `numba`
    3. С использованием векторизованных операций и методов массивов `numpy`, но без использования `numba`
    4. C использованием векторизованных операций и методов массивов `numpy` и `numba`
    
Измерьте время выполнения каждой из реализаций.

Замечание: удалите из выборки отзывы с нулевым рейтингом.


In [106]:
# Удаление отзывов с 0 рейтингом
reviews_without_zero_r = reviews.loc[reviews['rating'] >0]
reviews_without_zero_r.rating.value_counts()

5    91361
4    20866
3     4635
2     1628
1     1401
Name: rating, dtype: int64

In [75]:
reviews_without_zero_r.head()

,user_id,recipe_id,date,rating,review
0,21752,57993,2003-05-01,5,Last week whole sides of frozen salmon fillet ...
1,431813,142201,2007-09-16,5,So simple and so tasty! I used a yellow capsi...
2,400708,252013,2008-01-10,4,"Very nice breakfast HH, easy to make and yummy..."
3,2001852463,404716,2017-12-11,5,These are a favorite for the holidays and so e...
4,95810,129396,2008-03-14,5,Excellent soup! The tomato flavor is just gre...


In [107]:
# Средний рейтинг по отзывам
grouped_df = reviews_without_zero_r.groupby("recipe_id")
mean_r = grouped_df.mean()
mean_r = mean_r.reset_index().drop(['user_id'],axis=1)
mean_r.head()

,recipe_id,rating
0,48,2.000000
1,55,4.750000
2,66,4.944444
3,91,4.750000
4,94,5.000000


In [89]:
mean_r.shape[0]

27440

In [122]:
temporal = reviews_without_zero_r.loc[(reviews_without_zero_r['recipe_id'] == 536729) ]
temporal.head()

,user_id,recipe_id,date,rating,review
8673,482376,536729,2018-09-03,5,Awesome! And fun! What a great combination thi...
64742,1925885,536729,2018-08-16,5,"Loved this simple salad! DH saw it and said, &..."
90947,128473,536729,2018-08-28,5,Loved this salad sheepdoc. It was amazing. The...
111833,226863,536729,2018-10-06,4,Colors are great in this salad. For those that...


**А.Без использования векторизованных операций и методов массивов numpy и без использования numba**

In [186]:
recipe_536729 = list(reviews_without_zero_r.loc[(reviews_without_zero_r['recipe_id'] == 536729)].rating)
mean_536729 = int(mean_r.loc[(mean_r['recipe_id'] == 536729) ].rating)

In [126]:
def mape_for_all(data, mean_rating):
    mapes = {}
    for i, row in mean_rating.iterrows():
        recipe = int(row.recipe_id)
        mean_r = int(row.rating)
        temporal = reviews_without_zero_r.loc[(reviews_without_zero_r['recipe_id'] == recipe)]
        n = temporal.shape[0]
        delta = 0
        
        abs_er = 0 
        #для каждого рецепта считаем по формуле МАРЕ
        for j,row2 in temporal.iterrows():
             abs_er +=  (abs(row2.rating - mean_r)/row2.rating)
        mape = abs_er/n
        mapes[recipe] = mape
    return mapes   
            

In [187]:
def mape_a(data, mean_rating):
    n = len(data)
    delta = 0
    abs_er = 0
    for i in range(len(data)):
        abs_er += (abs(data[i]-mean_rating)/data[i])
    mape = abs_er/n
    return mape

In [188]:
%%timeit
mape_a(recipe_536729, mean_536729)

929 ns ± 7.14 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [131]:
%%timeit
mapes = mape_a(reviews, mean_r)

28.7 s ± 346 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


**Б.Без использования векторизованных операций и методов массивов numpy, но с использованием numba**

In [136]:
import numba

In [181]:
recipe_536729 = list(reviews_without_zero_r.loc[(reviews_without_zero_r['recipe_id'] == 536729)].rating)
mean_536729 = int(mean_r.loc[(mean_r['recipe_id'] == 536729) ].rating)


4

In [183]:
@numba.njit
def mape_b(data, mean_rating):
    n = len(data)
    delta = 0
    abs_er = 0
    for i in range(len(data)):
        abs_er += (abs(data[i]-mean_rating)/data[i])
    mape = abs_er/n
    return mape
        

In [184]:
%%timeit
mapes2 = mape_b(recipe_536729,mean_536729)

/Users/dgridnev/opt/anaconda3/lib/python3.7/site-packages/numba/ir_utils.py:2041: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'data' of function 'mape_b'.

For more information visit http://numba.pydata.org/numba-doc/latest/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types

File "<ipython-input-183-0d7d5a40020a>", line 2:
@numba.njit
def mape_b(data, mean_rating):
^

  warnings.warn(NumbaPendingDeprecationWarning(msg, loc=loc))


The slowest run took 4.70 times longer than the fastest. This could mean that an intermediate result is being cached.
17 µs ± 13.5 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


**C.С использованием векторизованных операций и методов массивов numpy, но без использования numba**

In [189]:
import numpy as np

In [200]:
def mape_c(data, mean_rating):
    mape = np.mean(np.abs(data-mean_rating)/data)
    return mape

In [205]:
%%timeit
mapes2 = mape_c(np.array(recipe_536729),mean_536729)

11.4 µs ± 455 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


**D.C использованием векторизованных операций и методов массивов numpy и numba**

In [203]:
@numba.njit
def mape_d(data, mean_rating):
    mape = np.mean(np.abs(data-mean_rating)/data)
    return mape

In [206]:
%%timeit
mapes4 = mape_d(np.array(recipe_536729),mean_536729)

The slowest run took 10.79 times longer than the fastest. This could mean that an intermediate result is being cached.
4.74 µs ± 6.21 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
